In [3]:
import numpy as np
import pandas as pd
from Classes import Simulation_Team
from Simulate_Season import simulate_season

# Team divisions (important for tiebreaking and seeding)
divisions = {"Bills": "AFC East", "Dolphins": "AFC East", "Jets": "AFC East", "Patriots": "AFC East", "Bengals": "AFC North", "Browns": "AFC North", "Ravens": "AFC North", "Steelers": "AFC North", "Colts": "AFC South", "Jaguars": "AFC South", "Texans": "AFC South", "Titans": "AFC South", "Broncos": "AFC West", "Chargers": "AFC West", "Chiefs": "AFC West", "Raiders": "AFC West", "Cowboys": "NFC East", "Eagles": "NFC East", "Football Team": "NFC East", "Giants": "NFC East", "Bears": "NFC North", "Lions": "NFC North", "Packers": "NFC North", "Vikings": "NFC North", "Buccaneers": "NFC South", "Falcons": "NFC South", "Panthers": "NFC South", "Saints": "NFC South", "49ers": "NFC West", "Cardinals": "NFC West", "Rams": "NFC West", "Seahawks": "NFC West"}
# Season schedule as a list of dictionaries. A key-value pair in a dictionary is key = matchup, value = winning team / TIE / empty string
reg_season_schedule = [
    {"Cowboys at Buccaneers": "Buccaneers", "Jaguars at Texans": "Texans", "Chargers at Football Team": "Chargers", "Seahawks at Colts": "Seahawks", "Jets at Panthers": "Panthers", "Vikings at Bengals": "Bengals", "Cardinals at Titans": "Cardinals", "49ers at Lions": "49ers", "Steelers at Bills": "Steelers", "Eagles at Falcons": "Eagles", "Browns at Chiefs": "Chiefs", "Packers at Saints": "Saints", "Broncos at Giants": "Broncos", "Dolphins at Patriots": "Dolphins", "Bears at Rams": "Rams", "Ravens at Raiders": "Raiders"},
    {"Giants at Football Team": "Football Team", "Patriots at Jets": "Patriots", "Broncos at Jaguars": "Broncos", "Bills at Dolphins": "Bills", "49ers at Eagles": "49ers", "Rams at Colts": "Rams", "Raiders at Steelers": "Raiders", "Bengals at Bears": "Bears", "Texans at Browns": "Browns", "Saints at Panthers": "Panthers", "Vikings at Cardinals": "Cardinals", "Falcons at Buccaneers": "Buccaneers", "Titans at Seahawks": "Titans", "Cowboys at Chargers": "Cowboys", "Chiefs at Ravens": "Ravens", "Lions at Packers": "Packers"},
    {"Panthers at Texans": "Panthers", "Colts at Titans": "Titans", "Falcons at Giants": "Falcons", "Chargers at Chiefs": "Chargers", "Bengals at Steelers": "Bengals", "Bears at Browns": "Browns", "Ravens at Lions": "Ravens", "Saints at Patriots": "Saints", "Cardinals at Jaguars": "Cardinals", "Football Team at Bills": "Bills", "Jets at Broncos": "Broncos", "Dolphins at Raiders": "Raiders", "Seahawks at Vikings": "Vikings", "Buccaneers at Rams": "Rams", "Packers at 49ers": "Packers", "Eagles at Cowboys": "Cowboys"},
    {"Jaguars at Bengals": "Bengals", "Titans at Jets": "Jets", "Chiefs at Eagles": "Chiefs", "Panthers at Cowboys": "Cowboys", "Giants at Saints": "Giants", "Browns at Vikings": "Browns", "Lions at Bears": "Bears", "Texans at Bills": "Bills", "Colts at Dolphins": "Colts", "Football Team at Falcons": "Football Team", "Seahawks at 49ers": "Seahawks", "Cardinals at Rams": "Cardinals", "Steelers at Packers": "Packers", "Ravens at Broncos": "Ravens", "Buccaneers at Patriots": "Buccaneers", "Raiders at Chargers": "Chargers"},
    {"Rams at Seahawks": "Rams", "Jets at Falcons": "Falcons", "Lions at Vikings": "Vikings", "Saints at Football Team": "Saints", "Patriots at Texans": "Patriots", "Dolphins at Buccaneers": "Buccaneers", "Packers at Bengals": "Packers", "Broncos at Steelers": "Steelers", "Eagles at Panthers": "Eagles", "Titans at Jaguars": "Titans", "Browns at Chargers": "Chargers", "Bears at Raiders": "Bears", "49ers at Cardinals": "Cardinals", "Giants at Cowboys": "Cowboys", "Bills at Chiefs": "Bills", "Colts at Ravens": ""},
    {"Buccaneers at Eagles": "Buccaneers", "Dolphins at Jaguars": "Jaguars", "Chiefs at Football Team": "Chiefs", "Rams at Giants": "Rams", "Texans at Colts": "Colts", "Bengals at Lions": "Bengals", "Packers at Bears": "Packers", "Chargers at Ravens": "Ravens", "Vikings at Panthers": "Vikings", "Cardinals at Browns": "Cardinals", "Raiders at Broncos": "Raiders", "Cowboys at Patriots": "Cowboys", "Seahawks at Steelers": "Steelers", "Bills at Titans": "Titans"},
    {"Broncos at Browns": "Browns", "Panthers at Giants": "Giants", "Jets at Patriots": "Patriots", "Chiefs at Titans": "Titans", "Football Team at Packers": "Packers", "Falcons at Dolphins": "Falcons", "Bengals at Ravens": "Bengals", "Lions at Rams": "Rams", "Eagles at Raiders": "Raiders", "Texans at Cardinals": "Cardinals", "Bears at Buccaneers": "Buccaneers", "Colts at 49ers": "Colts", "Saints at Seahawks": "Saints"},
    {"Packers at Cardinals": "Packers", "Bengals at Jets": "Jets", "Titans at Colts": "Titans", "Rams at Texans": "Rams", "Steelers at Browns": "Steelers", "Eagles at Lions": "Eagles", "49ers at Bears": "49ers", "Panthers at Falcons": "Panthers", "Dolphins at Bills": "Bills", "Patriots at Chargers": "Patriots", "Jaguars at Seahawks": "Seahawks", "Football Team at Broncos": "Broncos", "Buccaneers at Saints": "Saints", "Cowboys at Vikings": "Cowboys", "Giants at Chiefs": "Chiefs"},
    {"Jets at Colts": "Colts", "Falcons at Saints": "Falcons", "Broncos at Cowboys": "Broncos", "Patriots at Panthers": "Patriots", "Vikings at Ravens": "Ravens", "Browns at Bengals": "Browns", "Bills at Jaguars": "Jaguars", "Texans at Dolphins": "Dolphins", "Raiders at Giants": "Giants", "Chargers at Eagles": "Chargers", "Packers at Chiefs": "Chiefs", "Cardinals at 49ers": "Cardinals", "Titans at Rams": "Titans", "Bears at Steelers": "Steelers"},
    {"Ravens at Dolphins": "Dolphins", "Bills at Jets": "Bills", "Buccaneers at Football Team": "Football Team", "Falcons at Cowboys": "Cowboys", "Saints at Titans": "Titans", "Jaguars at Colts": "Colts", "Lions at Steelers": "TIE", "Browns at Patriots": "Patriots", "Vikings at Chargers": "Vikings", "Panthers at Cardinals": "Panthers", "Eagles at Broncos": "Eagles", "Seahawks at Packers": "Packers", "Chiefs at Raiders": "Chiefs", "Rams at 49ers": "Rams"},
    {"Patriots at Falcons": "Patriots", "Saints at Eagles": "Eagles", "Dolphins at Jets": "Dolphins", "Football Team at Panthers": "Football Team", "Colts at Bills": "Colts", "Lions at Browns": "Browns", "49ers at Jaguars": "49ers", "Texans at Titans": "Texans", "Packers at Vikings": "Vikings", "Ravens at Bears": "Ravens", "Bengals at Raiders": "Bengals", "Cardinals at Seahawks": "Cardinals", "Cowboys at Chiefs": "Chiefs", "Steelers at Chargers": "Chargers", "Giants at Buccaneers": "Buccaneers"},
    {"Bears at Lions": "Bears", "Raiders at Cowboys": "Raiders", "Bills at Saints": "Bills", "Buccaneers at Colts": "Buccaneers", "Jets at Texans": "Jets", "Eagles at Giants": "Giants", "Panthers at Dolphins": "Dolphins", "Titans at Patriots": "Patriots", "Steelers at Bengals": "Bengals", "Falcons at Jaguars": "Falcons", "Chargers at Broncos": "Broncos", "Rams at Packers": "Packers", "Vikings at 49ers": "49ers", "Browns at Ravens": "Ravens", "Seahawks at Football Team": "Football Team"},
    {"Cowboys at Saints": "Cowboys", "Giants at Dolphins": "Dolphins", "Colts at Texans": "Colts", "Vikings at Lions": "Lions", "Eagles at Jets": "Eagles", "Broncos at Chiefs": "Chiefs", "Cardinals at Bears": "Cardinals", "Chargers at Bengals": "Chargers", "Buccaneers at Falcons": "Buccaneers", "Football Team at Raiders": "Football Team", "Jaguars at Rams": "Rams", "Ravens at Steelers": "Steelers", "49ers at Seahawks": "Seahawks", "Patriots at Bills": "Patriots"},
    {"Steelers at Vikings": "Vikings", "Cowboys at Football Team": "Cowboys", "49ers at Bengals": "49ers", "Jaguars at Titans": "Titans", "Seahawks at Texans": "Seahawks", "Raiders at Chiefs": "Chiefs", "Saints at Jets": "Saints", "Falcons at Panthers": "Falcons", "Ravens at Browns": "Browns", "Giants at Chargers": "Chargers",  "Lions at Broncos": "Broncos", "Bills at Buccaneers": "Buccaneers", "Bears at Packers": "Packers", "Rams at Cardinals": "Rams"},
    {"Chiefs at Chargers": "Chiefs", "Cowboys at Giants": "Cowboys", "Texans at Jaguars": "Texans", "Titans at Steelers": "Steelers", "Packers at Ravens": "Packers", "Cardinals at Lions": "Lions", "Bengals at Broncos": "Bengals", "Falcons at 49ers": "49ers", "Seahawks at Rams": "Rams", "Saints at Buccaneers": "Saints", "Vikings at Bears": "Vikings", "Jets at Dolphins": "Dolphins", "Football Team at Eagles": "Eagles", "Raiders at Browns": "Raiders", "Panthers at Bills": "Bills", "Patriots at Colts": "Colts"},
    {"49ers at Titans": "Titans", "Browns at Packers": "Packers", "Colts at Cardinals": "Colts", "Giants at Eagles": "Eagles", "Rams at Vikings": "Rams", "Bills at Patriots": "Bills", "Buccaneers at Panthers": "Buccaneers", "Jaguars at Jets": "Jets", "Lions at Falcons": "Falcons", "Chargers at Texans": "Texans", "Ravens at Bengals": "Bengals", "Bears at Seahawks": "Bears", "Steelers at Chiefs": "Chiefs", "Broncos at Raiders": "Raiders", "Football Team at Cowboys": "Cowboys", "Dolphins at Saints": "Dolphins"},
    {"Eagles at Football Team": "Eagles", "Panthers at Saints": "Saints", "Buccaneers at Jets": "Buccaneers", "Dolphins at Titans": "Titans", "Jaguars at Patriots": "Patriots", "Raiders at Colts": "Raiders", "Chiefs at Bengals": "Bengals", "Giants at Bears": "Bears", "Falcons at Bills": "Bills", "Cardinals at Cowboys": "Cardinals", "Texans at 49ers": "49ers", "Broncos at Chargers": "Chargers", "Rams at Ravens": "Rams", "Lions at Seahawks": "Seahawks", "Vikings at Packers": "Packers", "Browns at Steelers": "Steelers"},
    {"Cowboys at Eagles": "", "Packers at Lions": "", "Colts at Jaguars": "", "Football Team at Giants": "", "Bears at Vikings": "", "Panthers at Buccaneers": "", "Patriots at Dolphins": "", "Saints at Falcons": "", "Titans at Texans": "", "Steelers at Ravens": "", "Jets at Bills": "", "Bengals at Browns": "", "Chargers at Raiders": "", "49ers at Rams": "", "Chiefs at Broncos": "", "Seahawks at Cardinals": ""},
]

_teams = {}
team_records_sheet = pd.ExcelFile("Base/Excel Files/Team Records.xlsx").parse("Sheet1") # Get initial team records at starting point

# Create teams and players data for each team
for key in divisions:
    _teams[key] = Simulation_Team(key, year="2021 ", records=team_records_sheet[key])
    _teams[key].conference = divisions[key][0:3]
    _teams[key].division = divisions[key][4:]
    _teams[key].create_players_data()

# Add opponents lists for each team
for i in range(0, len(reg_season_schedule)):
    for game in reg_season_schedule[i]:
        road_team = game[0:game.index(" at")]
        home_team = game[game.index(" at") + 4:]
        if i == 0:
            _teams[road_team].opponents = np.reshape(np.append(_teams[road_team].opponents, [home_team, "R", i + 1], axis=0), (1, -1))
            _teams[home_team].opponents = np.reshape(np.append(_teams[home_team].opponents, [road_team, "H", i + 1], axis=0), (1, -1))
        else:
            _teams[road_team].opponents = np.append(_teams[road_team].opponents, [[home_team, "R", i + 1]], axis=0)
            _teams[home_team].opponents = np.append(_teams[home_team].opponents, [[road_team, "H", i + 1]], axis=0)

n = 2500 # Number of times to simulate the season
start_week = 18 # Starting point
for i in range(0, n):
    simulate_season(_teams, reg_season_schedule, team_records_sheet, start_week)
    print(i + 1)

# Create data frame containing data from the season simulation
df = pd.DataFrame([[round(100 * _teams[team].super_bowl_wins / n, 2) for team in _teams], [round(100 * _teams[team].conference_champions / n, 2) for team in _teams], [round(100 * _teams[team].division_round_apps / n, 2) for team in _teams], [round(100 * _teams[team].playoff_apps / n, 2) for team in _teams], [round(100 * _teams[team].division_wins / n, 2) for team in _teams], [round(np.mean(_teams[team].all_seasons_stats["PYDs"]), 2) for team in _teams], [round(np.mean(_teams[team].all_seasons_stats["PTDs"]), 2) for team in _teams], [round(np.mean(_teams[team].all_seasons_stats["INTs"]), 2) for team in _teams], [round(np.mean(_teams[team].all_seasons_stats["WINS"]), 2) for team in _teams]], columns=[_teams[team].name for team in _teams], index=["SB win %", "Conf. win %", "Div. round %", "Playoff %", "Div. win %", "PYDs", "PTDs", "INTs", "WINS"])
df.to_excel("Season Simulations/From Week " + str(start_week) + ".xlsx") # Write to excel
print("Done")

Chiefs   [11, 6, 0]
Bengals   [11, 6, 0]
Titans   [11, 6, 0]
Bills   [11, 6, 0]
Packers   [14, 3, 0]
Cardinals   [12, 5, 0]
Buccaneers   [12, 5, 0]
Cowboys   [11, 6, 0]
Patriots   [11, 6, 0]
Chargers   [10, 7, 0]
Raiders   [9, 8, 0]
Rams   [12, 5, 0]
49ers   [10, 7, 0]
Eagles   [10, 7, 0]
1
Done
